In [ ]:
# import library
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl


# style your matplotlib
mpl.style.use("seaborn-darkgrid")

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [ ]:
from tqdm import tqdm 


In [ ]:
# list of files in train folder
listfile=os.listdir("../input/mma-facial-expression/MMAFEDB/train")
listfile


In [ ]:
from skimage import io
image_m=[]  # convert menjadi array
label_m=[]
path="../input/mma-facial-expression/MMAFEDB/train"

for i in range(len(listfile)):
    # files in sub-folder
    file=os.listdir(path+"/"+listfile[i])

   # print(len(file))
    
      if(listfile[i]=="angry" or listfile[i]=="disgust" or listfile[i]=="fear"):
        for k in tqdm(range(len(file))):
            img=io.imread(path+"/"+listfile[i]+"/"+file[k])
            # convert image from BGR to RGB
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            image_m.append(img)
            label_m.append(i)

    else:
        # for other all 
        for k in tqdm(range(7000)):
            # read image
            img=io.imread(path+"/"+listfile[i]+"/"+file[k])
            # convert image from BGR to RGB
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            image_m.append(img)
            label_m.append(i)

In [ ]:
lab,b=np.unique(label_m,return_counts="True")
lab

In [ ]:
b

In [ ]:
import gc
gc.collect()

In [ ]:
# normalisasi data
# this wil scale image pixel from 0-255 to 0-1
image_m=np.array(image_m)/255.0
# convert label list menjadi array
label_m=np.array(label_m)

In [ ]:
Ekspresi={0:"Terkejut",1:"Takut",2:"Marah",3:"Netral",4:"Sedih",5:"Jijik",6:"Senang"}

In [ ]:
# shuffle and split image and labels menjadi train and test
from sklearn.model_selection import train_test_split
image_m,X_test,Y_train,Y_test=train_test_split(image_m,label_m,test_size=0.1)

gc.collect()

In [ ]:
# label menjadi Ekspresi
#['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']
Ekspresi={0:"Terkejut",1:"Takut",2:"Marah",3:"Netral",4:"Sedih",5:"Jijik",6:"Senang"}

In [ ]:
Ekspresi[2]

In [ ]:
def show_examples(image,label,idx):
    # create 4x4 figure
    fig,axes=plt.subplots(nrows=4,ncols=4,figsize=(16,16))
    # loop through each figure
    for idx_f,ax in zip(idx,axes.ravel()):
        # add image to figure
        ax.imshow(image[idx_f].squeeze(),cmap="gray")
        # add title to each figure
        ax.set_title(Ekspresi[label[idx_f]])
    plt.show()
idx=np.random.choice(16,16)
show_examples(image_m,Y_train,idx)

In [ ]:
# import all library yang dibutuhakan
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=applications.MobileNetV2(input_shape=(48,48,3),include_top=False,
                                         weights="imagenet")
pretrained_model.trainable=True
# add pretrained_model to model
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add layer
model.add(layers.Dense(1024, activation = 'relu'))
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.25))
model.add(layers.Dense(512, activation = 'relu'))
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.25))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

In [ ]:
#compile model

model.compile(optimizer=Adam(0.0001),loss="mean_squared_error",metrics=["mae"])

In [ ]:
cp="trained_model/model"

In [ ]:
# create a model checkpoint to save model
model_cp=tf.keras.callbacks.ModelCheckpoint(filepath=cp,
                                                   monitor="val_mae",
                                                   save_best_only=True,
                                                   save_weights_only=True,
                                                   mode="auto")
# this checkpoint save model when val_mae is lower then best val_mae                                              

In [ ]:
# now we will define learning rate reducer 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,
                                              monitor="val_mae",
                                              mode="auto",
                                              cooldown=0,
                                              patience=5,
                                              verbose=1,
                                              min_lr=1e-6)


In [ ]:
EPOCHS=300
BATCH_SIZE=32
# start training
history=model.fit(image_m,Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_cp])
#run

In [ ]:
# after training is finished 
# load best model
model.load_weights(cp)

In [ ]:
prediction_val=model.predict(X_test,batch_size=BATCH_SIZE)


In [ ]:
# prediction value 
prediction_val[:10]

In [ ]:
# original value
Y_test[:10]

In [ ]:
# save model untuk digunakan menguji model
model.save("model.h5")
print("Saved model to disk")

In [ ]:
# save and convert model to tensorflow lite model 
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

#save model 
with open("model.tflite","wb") as f:
    f.write(tflite_model)